[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mckinsey/vizro/blob/docs/add_colab_link/vizro-ai/examples/colab_example_dashboard.ipynb)

# Use Vizro-AI to generate Vizro dashboard

In [ ]:
# @title 1. Install Vizro-AI
#@markdown (TODO: change to !pip install vizro-ai when this PR is ready to merge)
# !pip install vizro-ai
!pip install "vizro-ai @ git+https://github.com/mckinsey/vizro.git@main#subdirectory=vizro-ai"

In [2]:
# @title 2. Secure your LLM API key as environment variable
#@markdown - click the `Secrets` button on the left
#@markdown - click `Add new secret`
#@markdown - insert "OPENAI_API_KEY" under `Name`
#@markdown - insert the API key value under `Value`
#@markdown - [optional] you might also need to insert "OPENAI_BASE_URL" and its values, based on the requirements from your API key provider.
#@markdown - click the `Notebook access` toggle to make it available for the notebook run

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = userdata.get("OPENAI_BASE_URL") # optional

In [ ]:
# @title ## 3. Input your dashboard question

# @markdown #### Instructions:
user_input = 'Create a gdp dashboard with 1 page. use a bar chart to show gdp per continent. add a filter to filter data by country' # @param {type:"string", placeholder:"Enter the requirements for the dashboard"}

print(user_input)

In [ ]:
# @title 4. Upload your CSV data
#@markdown - click `Choose Files` if you'd like to upload and use your own data
#@markdown - or click `Cancel upload` and use the default example data for initial exploration

import pandas as pd
from plotly.express.data import gapminder
from google.colab import files

uploaded = files.upload()

dfs = []

if not uploaded:
    print("No files uploaded. Use the plotly gapminder dataset as an example.")
    df = gapminder()
    dfs.append(df)
else:
    for fn in uploaded.keys():
        print('User uploaded file "{name}"'.format(
            name=fn, length=len(uploaded[fn])))

        df_uploaded = pd.read_csv(fn)
        dfs.append(df_uploaded)

In [ ]:
# @title 5. Choose the LLM model

LLM = 'gpt-4o' # @param ["gpt-4o", "gpt-3.5-turbo", "gpt-4-turbo"]

print(f"Selected LLM: {LLM}")

In [ ]:
# @title 6. Build the dashboard using Vizro-AI

import ipywidgets as widgets
from IPython.display import display

def run_code(button):
    from vizro_ai import VizroAI
    from vizro import Vizro

    Vizro._reset()
    vizro_ai = VizroAI(model=LLM)
    dashboard = vizro_ai.dashboard(dfs, user_input)

    vizro_dashboard = Vizro().build(dashboard)

    # Store the result in the button's object
    button1.result = vizro_dashboard

# Create and display the button
button1 = widgets.Button(description="Build Dashboard!")
display(button1)

# Link the button click to the function
button1.on_click(run_code)

In [ ]:
# @title 7. Render dashboard

# button1.result.run()

import ipywidgets as widgets
from IPython.display import display

def run_dashboard(button):
    button1.result.run()

# Create and display the button
button2 = widgets.Button(description="Run Dashboard!")
display(button2)

# Link the button click to the function
button2.on_click(run_dashboard)